## Setup & Importing packages 

In [16]:
# Importing required libraries

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Pinecone
from langchain.llms import OpenAI
from dotenv import load_dotenv
import langchain
import pinecone
import openai
import os

In [21]:
# Importing the required keys from our .env file
load_dotenv()

True

## Reading the PDF
Using the PyPDFDirectoryLoader

In [6]:
# Defining the function to read the PDF from our directory using the PyPDFDirectoryLoader package
def read_doc(directory):
    file_loader=PyPDFDirectoryLoader(directory)
    documents=file_loader.load()
    return documents

In [7]:
doc=read_doc('documents/')
len(doc)

32

## Dividing the PDF into Text Chunks
Using the RecursiveCharacterTextSplitter

In [9]:
## Now, we will divide the pdfs that we read into text chunks
def chunk_data(docs,chunk_size=800,chunk_overlap=50):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=chunk_size,chunk_overlap=chunk_overlap)
    doc=text_splitter.split_documents(docs)
    return docs

In [10]:
documents=chunk_data(docs=doc)
len(documents)

32

## Adding OpenAI Embeddings to text chunks

In [17]:
# Adding the Open AI API Key to use OpenAI Embeddings
embeddings=OpenAIEmbeddings(api_key=os.environ['OPENAI_API_KEY'])
embeddings

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x12752eb10>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x12ee606d0>, model='text-embedding-ada-002', dimensions=None, deployment='text-embedding-ada-002', openai_api_version='', openai_api_base=None, openai_api_type='', openai_proxy='', embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=set(), disallowed_special='all', chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None)

In [18]:
vectors=embeddings.embed_query("How are you?")
len(vectors)

1536

In [27]:
## Vector Search DB In Pinecone
pinecone.init(
    api_key="a04d5822-bbba-44bc-a078-5163e2535aeb",
    environment="gcp-starter"
)
index_name="langchainvector"

AttributeError: init is no longer a top-level attribute of the pinecone package.

Please create an instance of the Pinecone class instead.

Example:

    import os
    from pinecone import Pinecone, ServerlessSpec

    pc = Pinecone(
        api_key=os.environ.get("PINECONE_API_KEY")
    )

    # Now do stuff
    if 'my_index' not in pc.list_indexes().names():
        pc.create_index(
            name='my_index', 
            dimension=1536, 
            metric='euclidean',
            spec=ServerlessSpec(
                cloud='aws',
                region='us-west-2'
            )
        )

